In [1]:
# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#Resolvendo problema do chromeDrive
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# Pandas
import pandas as pd
# RegEx / Regular Expressions
import re
# Operating system
import os
import time

In [2]:
case = 'https://loja.caseih.com.br/search?text='
nh = 'https://loja.newhollandag.com.br/search?text='
notFound = pd.DataFrame()
notFound["Produto"] = ''
notFound['Motivo'] = ''
cwd = os.getcwd()

In [3]:
query = pd.read_csv('produtos_bot_yury_not_vv.txt', encoding = 'latin1', sep=',')
df_query = pd.DataFrame(query)
df_query.drop('IDX', axis=1, inplace=True)

In [4]:
def contains_number(string):
    return any(char.isdigit() for char in string)

In [5]:
def findOEM(text):
    oem = []
    if 'VV' in text:
        indice = text.find('VV')
        text = text.replace(text[indice:], '')
    if 'MCL' in text:
        cod_mcl = text.find('MCL')
        copy_text = text
        text = text[:cod_mcl]
        cod_mcl = copy_text[cod_mcl:]
        cod_mcl = cod_mcl.replace(' ', '')
        if contains_number(cod_mcl):
            if len(cod_mcl) > 5:
                oem.append(cod_mcl)
    if not contains_number(text):
        return oem
    num = re.search(r'\d+', text)
    indice = text.find(num.group())
    lastspace = text.rfind(' ', 0, indice)
    remover = [text[:lastspace], '.', '-']
    for rm in remover:
        text = text.replace(rm, '')
    text = text.replace(' ', '/')
    text = text.split('/')
    for txt in text:
        if contains_number(txt):
            if len(txt) > 5:
                if not contains_number(txt[-1]):
                    txt = txt.replace(txt[-1], '')
                oem.append(txt)
    return oem

In [6]:
def nh_case(nav, cod, index, marca):
    try:
        nav.find_element(By.ID, 'product-not-found')
        notFound.loc[len(notFound)] = [index, f'Não encontrado no site {marca}']
        return
    except Exception as e:
        btn = nav.find_element(By.CLASS_NAME, 'product-card')
        link = btn.find_element(By.CLASS_NAME, 'stretched-link').get_attribute("href")
        nav.get(link)
        nome = nav.find_element(By.CLASS_NAME, 'product-title__product-name').text
        codigo = nav.find_element(By.CLASS_NAME, 'product-code').text
        elimg = nav.find_elements(By.CLASS_NAME, 'owl-carousel__images')
        src = []
        for img in elimg:
            img = img.get_attribute('src')
            if 'loja' not in img:
                src.append(img)
        src = list(dict.fromkeys(src))
        keys = []
        values = []
        keys.append('Nome: ')
        values.append(nome)
        keys.append('Cód. Cotripal: ')
        values.append(index)
        keys.append('OEM: ')
        values.append(codigo)
        for k in nav.find_elements(By.CLASS_NAME, 'product-specifications-item__title'):
            keys.append(k.text)
        for v in nav.find_elements(By.CLASS_NAME, 'product-specifications-item__value'):
            values.append(v.text)
        dict_infos = dict(zip(keys, values))
        dict_infos = str(dict_infos)
        dict_infos = dict_infos.replace("'", '"')
        images = {str(index) : src}
        images = str(images)
        images = images.replace("'", '"')
        with open(f'{cwd}/NhCase/prods.json', 'a') as teste:
            teste.write(dict_infos)
            teste.write('\n')
        with open(f'{cwd}/NhCase/images.json', 'a') as teste:
            teste.write(images)
            teste.write('\n')
def nhCase(nav, cod, index):
    try:
        nav.get(str(nh + cod))
        nh_case(nav, cod, index, 'New Holland')
        nav.get(str(case + cod))
        nh_case(nav, cod, index, 'Case')
    except:
        pass

In [7]:
chrome_options = webdriver.ChromeOptions()
servico = Service(ChromeDriverManager().install())
#chrome_options.add_argument('headless')
nav = webdriver.Chrome(options=chrome_options, service=servico)

In [8]:
produtos = dict(zip(df_query['ET0100_PRODUTO'], df_query['ET0100_NOME']))
for ID, cod in produtos.items():
    cods = findOEM(cod)
    for item in cods:
        nhCase(nav, item, ID)

In [9]:
# nhCase(nav,'84257511', '102431870')

In [10]:
display(notFound)

,Produto,Motivo
0,101011714,Não encontrado no site New Holland
1,101011714,Não encontrado no site Case
2,100276657,Não encontrado no site New Holland
3,100276657,Não encontrado no site Case
4,101066264,Não encontrado no site New Holland
...,...,...
160,101167114,Não encontrado no site Case
161,101053066,Não encontrado no site New Holland
162,101053066,Não encontrado no site Case
163,101053066,Não encontrado no site New Holland


In [11]:
notFound.to_excel(f'{cwd}/NhCase/notFound.xlsx')